# Define a new `LAMMPS` simulator with the implicit derivative functionality
`LammpsImplicitDer` is the core parent class that contains the implicit derivative implementations and the LAMMPS simulator. Any LAMMPS system class should inherit from this class.

In [1]:
import numpy as np
from lammps_implicit_der import LammpsImplicitDer, SNAP

## Example how to create a LAMMPS simulation based on `LammpsImplicitDer` class

In [12]:
class Bcc(LammpsImplicitDer):
    def __init__(self):
        super().__init__()

        # Load the SNAP potential
        self.pot = SNAP.from_files('W.snapcoeff', data_path='../lammps_implicit_der/data_files/')

bcc = Bcc()

None
Running LAMMPS with the following arguments:
-screen none -log none


## Setup a simple bcc system with a supercell of `ncell_x` unit cells

In [3]:
ncell_x = 2
bcc.lmp.commands_string(f"""
    clear

    # Disable atom sorting
    atom_modify map array sort 0 0.0

    # Periodic boundary conditions
    boundary p p p

    units metal

    # Origin at 1% of the lattice spacing to avoid numerical issues on the boundaries
    lattice bcc 3.13 origin 0.01 0.01 0.01

    # Define a region C as block from 0 to ncell_x lattice spacings in each direction
    region C block 0 {ncell_x} 0 {ncell_x} 0 {ncell_x} units lattice

    # Create the simulation box with one atom type
    create_box 1 C

    # Create atoms in the box
    create_atoms 1 region C

    # Set the mass
    mass * 45.0
    """)

## Setup the SNAP potential

In [4]:
bcc.setup_snap_potential()
print(bcc.pot.elem_list)
print(bcc.pot.Theta_dict)
print(bcc.pot.snapparam_dict)

['W']
{'W': {'elem_params': {'radius': 0.5, 'weight': 1.0}, 'beta0': -5.30602553034076, 'Theta': array([ 2.59171211e-02,  2.29459519e-02,  5.42267608e-02, -1.51585786e-02,
        2.71030824e-01,  6.53694819e-02,  3.47579854e-04, -6.36676650e-03,
        1.26170571e-01,  1.00532763e-01,  5.01645978e-02,  1.00570965e-01,
        2.52031414e-03, -2.00524259e-02,  2.86825036e-02, -6.64412013e-03,
        7.04211807e-02,  9.24840913e-02,  9.69625335e-03,  3.52187019e-02,
        9.47117594e-04,  9.90517179e-03, -1.69731525e-03,  6.53240492e-02,
        1.12288898e-01,  7.01812880e-02,  6.09595356e-02, -2.74980622e-02,
        3.72170044e-02,  5.02702975e-02,  7.06345397e-03, -1.07971357e-02,
       -2.07582090e-02,  2.41023973e-02,  9.68293034e-02,  2.74989592e-03,
        1.15041120e-02,  5.71673560e-02,  6.55507839e-03,  3.17319182e-02,
        2.12856497e-02,  1.31183642e-02,  7.55440708e-04,  7.52558175e-03,
        4.17081670e-02, -6.30345924e-03,  1.33449964e-03, -1.64835264e-04,
   

## Energy minimization

In [5]:
bcc.minimize_energy()

Minimizing energy with the following parameters:
ftol: 1e-08, maxiter: 1000, maxeval: 1000, algo: cg


## Compute the descriptors and their derivatives

In [6]:
bcc.run_init()

Minimizing energy with the following parameters:
ftol: 1e-08, maxiter: 1000, maxeval: 1000, algo: cg
Number of atoms: 16, largest force value: 8.549e-15, force norm: 3.312e-14


In [7]:
print(bcc.X_coord.shape)
print(bcc.species.shape)

(48,)
(16,)


In [8]:
print(bcc.timings)

==================== TIMING SUMMARY: ImplicitDer-LAMMPS ==============
                                     Tag  Time (s)     Calls     
----------------------------------------------------------------------
         ▷----------------initialization  0.33815      1         
         ▷----------------------run_init  0.04705      1         
         ▷---------------minimize_energy  0.03680      2         
         ▷-------------------gather_D_dD  0.00695      1         
         ▷------------------compute_D_dD  0.00505      1         




## Compute implicit derivative

In [9]:
bcc.scatter_coord()
bcc.gather_D_dD()
dX_dTheta = bcc.implicit_derivative(method='sparse')
print(dX_dTheta)

  0%|          | 0/55 [00:00<?, ?it/s]

100%|██████████| 55/55 [00:00<00:00, 105.45it/s]

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## Perturb the positions

In [10]:
bcc.X_coord = bcc.X_coord + np.random.normal(0, 0.001, bcc.X_coord.shape)
bcc.scatter_coord()
bcc.gather_D_dD()
dX_dTheta = bcc.implicit_derivative(method='sparse', maxiter=50)
print(dX_dTheta)

100%|██████████| 55/55 [00:15<00:00,  3.65it/s]

[[ 2.71142810e-04 -3.78314161e-03 -2.40389408e-03 ... -9.37221650e-04
   6.07927227e-03  3.45532559e-03]
 [ 1.62735028e-03 -3.04811482e-04 -2.81034324e-04 ... -1.21364558e-03
  -2.55709707e-04  8.02585187e-04]
 [-5.99441931e-04 -3.27270808e-04 -2.38699325e-04 ...  2.65993049e-05
   7.11683955e-04 -1.93513925e-04]
 ...
 [ 1.10530583e-02 -1.02441387e-02  5.58627922e-03 ... -2.07325148e-02
   8.20729858e-03 -1.78669058e-02]
 [ 2.16639282e-02  4.72151520e-03  2.75324823e-03 ... -7.68639750e-03
  -3.28302800e-02 -2.40947354e-03]
 [ 6.29889454e-03 -6.70330836e-05  6.06788184e-03 ...  1.82669379e-03
  -1.42709417e-03 -4.23475012e-04]]


In [11]:
print(bcc.A_hard)

[[-0.53548859  0.20788133  0.1464997  ... -0.00797018 -0.08613304
  -0.05300918]
 [-0.00378981 -0.00707492  0.00333237 ... -0.02127212 -0.04197775
   0.09063668]
 [ 0.00926248  0.01053152 -0.00177658 ... -0.02894891  0.00439351
   0.02763794]
 ...
 [ 0.01127219  0.13936971 -0.01325284 ... -0.06664941 -0.0904004
   0.10323431]
 [ 0.01601187  0.04278328  0.07669818 ... -0.26126275  0.08816138
  -0.3536606 ]
 [ 0.00158589 -0.01683313  0.06320244 ... -0.01324029 -0.01038556
  -0.06258863]]
